In [78]:
from bs4 import BeautifulSoup
import re
import requests
import pandas as pd

# Todo:
Read url links from scraper_one_results

In [79]:
urls = [
    "https://en.wikipedia.org//wiki/ONE_Fight_Night_8",
    "https://en.wikipedia.org//wiki/2013_in_ONE_Championship",
    "https://en.wikipedia.org//wiki/2017_in_ONE_Championship",
    "https://en.wikipedia.org//wiki/2012_in_ONE_Championship",
    "https://en.wikipedia.org//wiki/2021_in_ONE_Championship",
]

# Todo:
add function to extract event card from df

In [80]:
def getData(url):
    dfs = []
    soup = BeautifulSoup(requests.get(url).text)
    header_patterns = ["one championship:", "road to one", "one on", "one fighting championship"]
    pattern_match = lambda x: len([_ for _ in header_patterns if _ in x.lower()]) > 0

    # event_headers = soup.find_all(lambda tag: tag.name == "h2" and "one" in tag.text.lower() and "cancelled" not in tag.text.lower())
    event_headers = [header for header in soup.find_all(lambda tag: tag.name == "h2") if pattern_match(header.text)]

    if not event_headers:
        event_headers = soup.find_all(lambda tag: tag.name == "h1" and "one" in tag.text.lower() and "cancelled" not in tag.text.lower())

    table_classes = [_.get("class") for _ in event_headers[0].find_all_next("table")]

    table_class = "wikitable" if "toccolours" not in [x for xs in list(filter(lambda x: x!=None, table_classes)) for x in xs] else "toccolours"

    for event in event_headers:

        event_name = event.text

        df = pd.read_html(str(event.find_next("table", class_ = table_class)))[0]
        df = df.assign(event_name = event_name)
        dfs.append(df)
    return pd.concat(dfs, ignore_index=True)

# Todo:
Convert section to pyspark

In [81]:
df = pd.concat([getData(url) for url in urls], ignore_index=True)

In [52]:
raw_df = pd.read_csv(r"C:\Development\ultimateNakMuay\data\raw\wiki_events_onefc.csv")

In [100]:
event_exists = lambda x: (
    (x.replace("[edit]", "") in raw_df["Event"].to_list()) or
    (x.replace("[edit]", "").replace(" Championship", "") in raw_df["Event"].to_list()) or
    (x.split(":")[-1].replace("[edit]", "").strip() in [_.split(":")[-1].strip() for _ in raw_df["Event"].to_list()]) or
    (x.split(":")[0].replace("[edit]", "").strip() in [_.split(":")[0].strip() for _ in raw_df["Event"].to_list()])
)

In [101]:
set([event for event in df["event_name"].to_list() if event_exists(event) == False])

set()

In [99]:
raw_df[raw_df["Event"].str.contains("ONE Fight Night 8")]

,#,Event,Date,Venue,Location,Attendance
17,236,ONE Fight Night 8: Superlek vs. Williams,"March 25, 2023",Singapore Indoor Stadium,"Kallang, Singapore",NaN
